# Mask R-CNN - Detect on Prostate Dataset

This notebook shows how to use trained Mask R-CNN on prostate dataset for whole slide. You'd need a GPU, though, because the network backbone is a Resnet101, which would be slow to detect on a CPU.

The code of the Prostate dataset can be found in prostate.py.

## Import Module

In [ ]:
# import module from system lib
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scipy.io

In [ ]:
# import module from maskrcnn repo
from config import Config
import utils
import model as modellib
import visualize
from model import log
import prostate

%matplotlib inline

# Specify GPU to use
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Configurations

In [ ]:
# Specify the dir that store the prostate dataset
dataset_dir = os.path.join(os.path.dirname(os.getcwd()), "Data_Pre_Processing/cedars-224")
# We do 5-fold validation, specify which fold to be exclude for the current run
held_out_set = 4
# Featch the mean_pixel based on the training data (data exclude the held_out_set)
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
# Configuration
class DetectionConfig(prostate.ProstateConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_NMS_THRESHOLD = 1     
    MEAN_PIXEL = np.array(mean_pixel)
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_DIM = 512
    DETECTION_CROP = [128, 384, 128, 384] # [height_crop_start, height_crop_end, width_crop_start, width_crop_end]
    MODE = -1
detection_config = DetectionConfig()
detection_config.display()

## Notebook Figure Configure

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Create Model Graph and Loading Weights

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="detection", 
                          config=detection_config,
                          model_dir=MODEL_DIR)
# Get path to saved weights
# Either set a specific path, find a trained weights specified by epoch and held_out_set or find last trained weights
h5_filename = None # Specify the h5 filename here if you want to choose a specific file
epoch = -1

if h5_filename is not None:
    model_path = os.path.join(ROOT_DIR, ".h5 file name here")
elif epoch == -1:    
    model_path = model.find_last()[1]
else:
    try:
        model_path = model.find_specific(epoch = epoch, held_out_set = held_out_set)[1]
    except:
        model_path = model.find_specific(epoch = epoch)[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## Prepare the Dataset and Specify the Interested Image

In [ ]:
Random = False # whether to randomly choose the image
# Specify the image that is interested in
val_list = [0]
# val_list = [image for image in val_list if image not in exclude_list]
if Random:
    _, val_list = dataset_val.generator_patition(dataset_dir, held_out_set)    
    image_id = random.choice(val_list)
    val_list = [image_id]
    image_patch_num = np.random.randint(0, detection_config.MODE)

dataset_val = prostate.ProstateDataset()
dataset_val.load_prostate(dataset_dir, val_list, mode = detection_config.MODE)
dataset_val.prepare()

## Load image and ground truth data

In [ ]:
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_val.class_names, figsize=(8, 8))

## Run Detection

In [ ]:
results = model.detect([original_image], verbose=1)
r = results[0]
det_sementic_mask = r['sementic_mask']
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

In [ ]:
# Convert gt-instance mask to gt-sementic mask
gt_sementic_mask = utils.instance_2_sementic(gt_mask, gt_class_id)
# Visualization the sementic map
visualize.display_sementic(original_image, gt_sementic_mask['ATmask'],
                            figsize=(8, 8))
visualize.display_sementic(original_image, det_sementic_mask,
                            figsize=(8, 8))

## Evaluation

In [ ]:
# Compute confusion matrix
c_matrix = confusion_matrix(np.ravel(gt_sementic_mask['ATmask']), np.ravel(result_sementic))
c_matrix = utils.expand_c_matrix(c_matrix, detection_config.NUM_CLASSES)

# Compute mIOU based on confusion matrix
mIOU, IOU, below_th = compute_mIOU(c_matrix, th = 0.5)
print('Confusion Matrix:\n', c_matrix)
print(' mIOU:', mIOU, '\n', 
      'IOU for each class:', IOU, '\n',
      'Below_th:', below_th)